# PySpark Huggingface Inferencing
### Sentence Transformers

From: https://huggingface.co/sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

#Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']


#Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)

In [ ]:
embedding

## PySpark

## Inference using Spark ML Model
Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

In [ ]:
import sparkext

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(100)

In [ ]:
df.show(truncate=80)

In [ ]:
my_model = sparkext.huggingface.SentenceTransformerModel(model) \
                .setInputCol("lines") \
                .setOutputCol("embedding")

In [ ]:
embeddings = my_model.transform(df)

In [ ]:
%%time
results = embeddings.collect()

In [ ]:
embeddings.show(truncate=60)

## Inference using Spark DL UDF
Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

### Using model instance on driver

In [ ]:
from pyspark.sql.functions import col
from sparkext.huggingface import sentence_transformer_udf

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

In [ ]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(100)

In [ ]:
df.schema

In [ ]:
encode = sentence_transformer_udf(model)

In [ ]:
embeddings = df.withColumn("encoding", encode(col("lines")))

In [ ]:
%%time
results = embeddings.collect()

In [ ]:
embeddings.show(truncate=60)

### Using model_id string on driver

In [ ]:
from pyspark.sql.functions import col
from sparkext.huggingface import sentence_transformer_udf

In [ ]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(100)

In [ ]:
encode = sentence_transformer_udf("paraphrase-MiniLM-L6-v2")

In [ ]:
embeddings = df.withColumn("encoding", encode(col("lines")))

In [ ]:
%%time
results = embeddings.collect()

In [ ]:
embeddings.show(truncate=60)

### Using model loader

In [ ]:
from pyspark.sql.functions import col
from sparkext.huggingface import sentence_transformer_udf

In [ ]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(100)

In [ ]:
def model_loader(model_name):
    from sentence_transformers import SentenceTransformer
    return SentenceTransformer(model_name)   

In [ ]:
encode = sentence_transformer_udf("paraphrase-MiniLM-L6-v2", model_loader=model_loader)

In [ ]:
embeddings = df.withColumn("encoding", encode(col("lines")))

In [ ]:
%%time
results = embeddings.collect()

In [ ]:
embeddings.show(truncate=60)

## Inference using Spark DL API
Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

In [1]:
from pyspark.ml.functions import batch_infer_udf
from pyspark.sql.functions import col, struct
from pyspark.sql.types import ArrayType, FloatType

In [2]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(100)

In [3]:
df.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   lines|
+------------------------------------------------------------------------------------------------------------------------+
|...But not this one! I always wanted to know "what happened" next. We will never know for sure what happened because ...|
|I found myself getting increasingly angry as this movie progressed.<br /><br />Basically, Dr. Crawford (Dennis Hopper...|
|The comparisons between the 1995 version and this are inevitable. Sadly, this version falls far short.<br /><br />The...|
|Doesn't anyone bother to check where this kind of sludge comes from before blathering on about its supposed revelatio...|
|Don't get me wrong, I love the TV series of League Of Gentlemen. It was funny, twisted and completely inspired. I was...|
|Made it through

In [4]:
def predict_batch_fn():
    import numpy as np
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
    def predict(inputs):
        flattened = np.squeeze(inputs).tolist()
        return model.encode(flattened)
    return predict

In [5]:
encode = batch_infer_udf(predict_batch_fn,
                         return_type=ArrayType(FloatType()),
                         batch_size=10)

In [6]:
embeddings = df.withColumn("encoding", encode(struct("lines")))

In [7]:
embeddings.show(truncate=60)

+------------------------------------------------------------+------------------------------------------------------------+
|                                                       lines|                                                    encoding|
+------------------------------------------------------------+------------------------------------------------------------+
|Hard up, No proper jobs going down at the pit, why not re...|[0.086341254, -0.0022546588, 0.102132246, -0.034549125, -...|
|To be fair, I didn't see a lot of this show. Probably bec...|[-0.007345509, -0.22671106, 0.18855843, -0.14904073, -0.0...|
|(This might have a spoiler)<br /><br />When I first start...|[0.077323586, 0.28863674, -0.058391068, -0.100140095, -0....|
|1st watched 3/17/2002 - 2 out of 10(Dir-Mario Pinzauti): ...|[-0.16619368, 0.16603407, -0.22564588, -0.18695478, -0.47...|
|I really didn't like this film~!!!! it was boring and did...|[0.10531771, -0.012788184, 0.14610296, 0.034648202, -0.02...|
|A wildl

In [8]:
%%time
results = embeddings.collect()

CPU times: user 16.9 ms, sys: 0 ns, total: 16.9 ms
Wall time: 5.45 s


In [9]:
spark.stop()